# Composite analysis

- aggregate energy over multiple wavenumbers to create time series for composite selection ("slow" and "fast" phase speeds are still defined separately for each wavenumber)
- analyse composite variance of 30-day means in addition to composite means

The significance assement can be performed using a t-test and an F-test, respectively, or by performing parametric bootstraps.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import xhistogram.xarray
import cmocean
import os

from scipy import stats
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

from heat_util import basic_metrics
from wave_util import construct_rolling_dataset, remove_climatology, compute_spectra, centroid, integral
from composite_util import composite_dates, parametric_bootstrap

work = os.environ.get('WORK')+'/'
plt.rcParams.update({'font.size': 14})

In [ ]:
client = Client()

client

In [ ]:
client.close()

## Load data for meridional wind analysis

In [ ]:
spectra = xr.open_dataarray(work+'wolfgang/spectra_30days_65-35N_wave5-8.nc')

In [ ]:
### Meridional wind

directory = work+'DATA/ERA5/eth/plev/'

files = [directory + f for f in os.listdir(directory) if f.startswith('era5_an_vwind_reg2_6h') and 
                                            (f.endswith('06.nc') or
                                             f.endswith('07.nc') or
                                             f.endswith('08.nc'))]
files.sort() 

n_valid_years = int(len(files)/3)

files = [files[i:i+3] for i in range(0,n_valid_years*3,3)]

selection = dict(lat=slice(90,0),plev=25000)

rolling = construct_rolling_dataset(files,selection=selection,n_per_window=30*4)

rolling['rolling'] = rolling['rolling'] + np.timedelta64(3,'h')

anomalies = remove_climatology(rolling)['var132']


wind = anomalies.mean('time').compute()

## Estimate composite-mean and variance

In [ ]:
t_statistic = lambda x, ref: x.mean('time') / x.std('time') * len(x['time'])**(1/2)

def bootstrap_mean(sample,population):
    
    sig = parametric_bootstrap(sample,population,parameter=t_statistic,
                               pvalue=0.05,nrandom=10000)
    
    return xr.Dataset(dict(mean=sample.mean('time'),sig=sig))

f_statistic = lambda x, ref: x.var('time') / ref.var('rolling')

def bootstrap_variance(sample,population):
    
    sig = parametric_bootstrap(sample,population,parameter=f_statistic,
                               pvalue=0.05,one_sided=True,nrandom=10000)
    
    return xr.Dataset(dict(var=sample.var('time'),sig=sig))

In [ ]:
slow = []
fast = []

# create time series for composite construction
for w in range(5,9):                       
    
    bound = centroid(spectra.sel(wavenumber=w).mean('rolling'))
    slow.append(integral(spectra.sel(wavenumber=w),bound.values,-30))
    fast.append(integral(spectra.sel(wavenumber=w),30,bound.values))   

slow = sum(slow)
fast = sum(fast)

# select composite
dates = composite_dates(slow.rename(rolling='time'),percentile=0.9)
slow = wind.sel(rolling=dates)

dates = composite_dates(fast.rename(rolling='time'),percentile=0.9)
fast = wind.sel(rolling=dates)

composite = xr.concat([slow.assign_coords(speed='slow'),fast.assign_coords(speed='fast')],dim='speed')

In [ ]:
# prepare array for plotting
cyclic_point = composite.sel(lon=0)
cyclic_point['lon'] = 360

composite = xr.concat([composite,cyclic_point],dim='lon')

In [ ]:
# estimate composite mean and variance
composite_mean = bootstrap_mean(composite,wind)
composite_variance = bootstrap_variance(composite,wind)

## Figure 2

In [ ]:
## Plotting

fig = plt.figure(figsize=(8,4))

# mean - slow

ax1 = fig.add_subplot(2,2,3,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow').where(composite_mean['sig'].sel(speed='slow')).plot.contourf(ax=ax1,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow').plot.contour(ax=ax1,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax1.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax1.set_title('')
ax1.set_extent([-180,180,0,90],ccrs.PlateCarree())


# mean - fast

ax2= fig.add_subplot(2,2,4,projection=ccrs.EqualEarth())

C1 = composite_mean['mean'].sel(speed='fast').where(composite_mean['sig'].sel(speed='fast')).plot.contourf(ax=ax2,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast').plot.contour(ax=ax2,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax2.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax2.set_title('')
ax2.set_extent([-180,180,0,90],ccrs.PlateCarree())

 

# variance - slow

ax3 = fig.add_subplot(2,2,1,projection=ccrs.EqualEarth())

composite_variance['var'].sel(speed='slow').where(composite_variance['sig'].sel(speed='slow')).plot.contourf(ax=ax3,levels=np.linspace(0,60,7),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.matter,
                                              extend='max',
                                              add_colorbar=False)

composite_variance['var'].sel(speed='slow').plot.contour(ax=ax3,levels=np.linspace(0,60,7),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.matter,
                                                  extend='max',
                                                  add_colorbar=False)


ax3.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax3.set_title('Amplified Slow',weight='bold')
ax3.set_extent([-180,180,0,90],ccrs.PlateCarree())


# mean - fast

ax4= fig.add_subplot(2,2,2,projection=ccrs.EqualEarth())

C2 = composite_variance['var'].sel(speed='fast').where(composite_variance['sig'].sel(speed='fast')).plot.contourf(ax=ax4,levels=np.linspace(0,60,7),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.matter,
                                              extend='max',
                                              add_colorbar=False)

composite_variance['var'].sel(speed='fast').plot.contour(ax=ax4,levels=np.linspace(0,60,7),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.matter,
                                                  extend='max',
                                                  add_colorbar=False)


ax4.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax4.set_title('Amplified Fast',weight='bold')
ax4.set_extent([-180,180,0,90],ccrs.PlateCarree())

# config figure

fig.subplots_adjust(0,0.1,1,1,0.1,0.15)

cbar = plt.colorbar(C1,ax=[ax1,ax2],orientation='horizontal',shrink=0.8,pad=0.05,aspect=30)
cbar.set_label(r'V250 mean anomaly [m s$^{-1}$]')

cbar = plt.colorbar(C2,ax=[ax3,ax4],orientation='horizontal',shrink=0.8,pad=0.05,aspect=30)
cbar.set_label(r'V250 30-day variance [m$^{2}$ s$^{-2}$]')


## Single wavenumber composite mean – Figure S2

In [ ]:
slow = []
fast = []

for w in spectra['wavenumber']:                       
    
    # create time series for composite construction
    bound = centroid(spectra.sel(wavenumber=w).mean('rolling'))
    
    slow.append(integral(spectra.sel(wavenumber=w),bound.values,-30))
    fast.append(integral(spectra.sel(wavenumber=w),30,bound.values))
    
slow = xr.concat(slow,dim='wavenumber')
fast = xr.concat(fast,dim='wavenumber')

In [ ]:
composite = []

for w in spectra['wavenumber']:
    dates = composite_dates(slow.sel(wavenumber=w).rename(rolling='time'),percentile=0.9)
    tmp_slow = wind.sel(rolling=dates).drop(('time','rolling','my_index'))
    
    dates = composite_dates(fast.sel(wavenumber=w).rename(rolling='time'),percentile=0.9)
    tmp_fast = wind.sel(rolling=dates).drop(('time','rolling','my_index'))

    composite.append(xr.concat([tmp_slow.assign_coords(speed='slow'),tmp_fast.assign_coords(speed='fast')],dim='speed'))
    
composite = xr.concat(composite,dim='wavenumber')

In [ ]:
composite_mean = bootstrap_mean(composite,wind)

In [ ]:
fig = plt.figure(figsize=(8,8))


# wavnumber 5 – slow

ax1 = fig.add_subplot(4,2,1,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow',wavenumber=5).where(composite_mean['sig'].sel(speed='slow',wavenumber=5)).plot.contourf(ax=ax1,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow',wavenumber=5).plot.contour(ax=ax1,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax1.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax1.set_title('Amplified Slow',weight='bold')
ax1.text(-0.05,0.5,'Wave-5',weight='bold',
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax1.transAxes)
ax1.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 5 – fast

ax2 = fig.add_subplot(4,2,2,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='fast',wavenumber=5).where(composite_mean['sig'].sel(speed='fast',wavenumber=5)).plot.contourf(ax=ax2,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast',wavenumber=5).plot.contour(ax=ax2,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax2.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax2.set_title('Amplified Fast',weight='bold')
ax2.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 6 – slow

ax3 = fig.add_subplot(4,2,3,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow',wavenumber=6).where(composite_mean['sig'].sel(speed='slow',wavenumber=6)).plot.contourf(ax=ax3,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow',wavenumber=6).plot.contour(ax=ax3,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax3.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax3.set_title('')
ax3.text(-0.05,0.5,'Wave-6',weight='bold',
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax3.transAxes)
ax3.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 6 – fast

ax4 = fig.add_subplot(4,2,4,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='fast',wavenumber=6).where(composite_mean['sig'].sel(speed='fast',wavenumber=6)).plot.contourf(ax=ax4,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast',wavenumber=6).plot.contour(ax=ax4,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax4.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax4.set_title('')
ax4.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 7 – slow

ax5 = fig.add_subplot(4,2,5,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow',wavenumber=7).where(composite_mean['sig'].sel(speed='slow',wavenumber=7)).plot.contourf(ax=ax5,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow',wavenumber=7).plot.contour(ax=ax5,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax5.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax5.set_title('')
ax5.text(-0.05,0.5,'Wave-7',weight='bold',
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax5.transAxes)
ax5.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 7 – fast

ax6 = fig.add_subplot(4,2,6,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='fast',wavenumber=7).where(composite_mean['sig'].sel(speed='fast',wavenumber=7)).plot.contourf(ax=ax6,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast',wavenumber=7).plot.contour(ax=ax6,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax6.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax6.set_title('')
ax6.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 8 – slow

ax7 = fig.add_subplot(4,2,7,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow',wavenumber=8).where(composite_mean['sig'].sel(speed='slow',wavenumber=8)).plot.contourf(ax=ax7,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow',wavenumber=8).plot.contour(ax=ax7,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax7.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax7.set_title('')
ax7.text(-0.05,0.5,'Wave-8',weight='bold',
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax7.transAxes)
ax7.set_extent([-180,180,0,90],ccrs.PlateCarree())


# wavnumber 8 – fast

ax8 = fig.add_subplot(4,2,8,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='fast',wavenumber=8).where(composite_mean['sig'].sel(speed='fast',wavenumber=8)).plot.contourf(ax=ax8,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast',wavenumber=8).plot.contour(ax=ax8,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax8.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax8.set_title('')
ax8.set_extent([-180,180,0,90],ccrs.PlateCarree())


# config figure

fig.subplots_adjust(0,0.1,1,1,0.1,0.15)

cbar = plt.colorbar(C1,ax=[ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8],orientation='horizontal',shrink=0.8,pad=0.05,aspect=30)
cbar.set_label(r'V250 mean anomaly [m s$^{-1}$]')


## Figure S3

In [ ]:
## Zonal wind

directory = work+'DATA/ERA5/eth/plev/'

files = [directory + f for f in os.listdir(directory) if f.startswith('era5_an_uwind_reg2_6h') and 
                                            (f.endswith('06.nc') or
                                             f.endswith('07.nc') or
                                             f.endswith('08.nc'))]
files.sort() 

n_valid_years = int(len(files)/3)

files = [files[i:i+3] for i in range(0,n_valid_years*3,3)]

selection = dict(lat=slice(90,0),plev=25000)

rolling = construct_rolling_dataset(files,selection=selection,n_per_window=30*4)
rolling['rolling'] = rolling['rolling'] + np.timedelta64(3,'h')

anomalies = remove_climatology(rolling)['var131']

uwind = anomalies.mean('time').compute()


In [ ]:
slow = []
fast = []

# create time series for composite construction
for w in range(5,9):                       
    
    bound = centroid(spectra.sel(wavenumber=w).mean('rolling'))
    slow.append(integral(spectra.sel(wavenumber=w),bound.values,-30))
    fast.append(integral(spectra.sel(wavenumber=w),30,bound.values))   

slow = sum(slow)
fast = sum(fast)

# select composite
dates = composite_dates(slow.rename(rolling='time'),percentile=0.9)
slow = uwind.sel(rolling=dates)

dates = composite_dates(fast.rename(rolling='time'),percentile=0.9)
fast = uwind.sel(rolling=dates)

composite = xr.concat([slow.assign_coords(speed='slow'),fast.assign_coords(speed='fast')],dim='speed')

In [ ]:
cyclic_point = composite.sel(lon=0)
cyclic_point['lon'] = 360

composite = xr.concat([composite,cyclic_point],dim='lon')

In [ ]:
composite_mean = bootstrap_mean(composite,uwind)

In [ ]:
# zonal mean – not removing a seasonally varying climatology

zonal = rolling.mean(('lon','time'))['var131'].compute()

In [ ]:
slow = []
fast = []

# create time series for composite construction
for w in range(5,9):                       
    
    bound = centroid(spectra.sel(wavenumber=w).mean('rolling'))
    slow.append(integral(spectra.sel(wavenumber=w),bound.values,-30))
    fast.append(integral(spectra.sel(wavenumber=w),30,bound.values))   

slow = sum(slow)
fast = sum(fast)

# select composite
dates = composite_dates(slow.rename(rolling='time'),percentile=0.9)
slow = zonal.sel(rolling=dates)

dates = composite_dates(fast.rename(rolling='time'),percentile=0.9)
fast = zonal.sel(rolling=dates)

composite = xr.concat([slow.assign_coords(speed='slow'),fast.assign_coords(speed='fast')],dim='speed')

In [ ]:
composite_zonal = bootstrap_mean(composite,zonal)

In [ ]:
## Plotting

fig = plt.figure(figsize=(8,4))

# slow mean map

ax1 = fig.add_subplot(2,2,1,projection=ccrs.EqualEarth())

composite_mean['mean'].sel(speed='slow').where(composite_mean['sig'].sel(speed='slow')).plot.contourf(ax=ax1,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='slow').plot.contour(ax=ax1,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax1.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax1.set_title('')
ax1.text(-0.05,0.6,'Amplfied Slow',weight='bold',#fontsize=12,
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax1.transAxes)
ax1.set_extent([-180,180,0,90],ccrs.PlateCarree())


# fast mean map

ax2 = fig.add_subplot(2,2,3,projection=ccrs.EqualEarth())

C1 = composite_mean['mean'].sel(speed='fast').where(composite_mean['sig'].sel(speed='fast')).plot.contourf(ax=ax2,levels=np.linspace(-3,3,13),
                                              transform=ccrs.PlateCarree(),
                                              cmap=cmocean.cm.delta,
                                              extend='both',
                                              add_colorbar=False)

composite_mean['mean'].sel(speed='fast').plot.contour(ax=ax2,levels=np.concatenate((np.linspace(-3,-0.5,6),np.linspace(0.5,3,6))),
                                                  transform=ccrs.PlateCarree(),
                                                  cmap=cmocean.cm.delta,
                                                  extend='both',
                                                  add_colorbar=False)


ax2.add_feature(cartopy.feature.LAND, zorder=0, facecolor='darkgray', edgecolor=None)
ax2.set_title('')
ax2.text(-0.05,0.4,'Amplfied Fast',weight='bold',#fontsize=12,
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax2.transAxes)
ax2.set_extent([-180,180,0,90],ccrs.PlateCarree())


# zonal mean

ax3 = fig.add_subplot(1,2,2)


ax3.plot([0,90],[0,0],color='#b0b0b0',linewidth=0.8)
ax3.plot([zonal.mean('rolling')['lat'][zonal.mean('rolling').argmax()],zonal.mean('rolling')['lat'][zonal.mean('rolling').argmax()]],
        [-3,3],color='#b0b0b0',linewidth=0.8)

l1 = (composite_zonal['mean'].sel(speed='slow')-zonal.mean('rolling')).plot(ax=ax3,linewidth=3)
l2 = (composite_zonal['mean'].sel(speed='fast')-zonal.mean('rolling')).plot(ax=ax3,linewidth=3,linestyle='dotted')

ax3.set_xlim(0,90)
ax3.set_ylim(-3,3)
ax3.set_xlabel('Latitude [°N]',fontsize=12,)
ax3.set_ylabel(r'Zonal-mean anomaly [m s$^{-1}$]',fontsize=12)
ax3.set_title('')
      
ax4 = ax3.twinx()

l3 = zonal.mean('rolling').plot(ax=ax4,color='grey')

ax4.set_ylim(-20,20)
ax4.set_ylabel(r'Clim. zonal-mean zonal wind [m s$^{-1}$]',fontsize=12)
ax4.set_title('')

plt.legend([*l1,*l2,*l3],['slow','fast','clim'],fontsize=12)


# config figure

fig.subplots_adjust(0,0.1,1,1,0.25,0.15)

cbar = plt.colorbar(C1,ax=[ax1,ax2],orientation='horizontal',shrink=0.8,pad=0.05,aspect=30)
cbar.set_label(r'U250 mean anomaly [m s$^{-1}$]')
